In [12]:
# Ignore warnings for this execution
import warnings
warnings.filterwarnings('ignore')

# 📦 Install required packages (run in Colab)
!pip install pathway bokeh panel --quiet

In [13]:
# 📁 Load and preprocess data
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, atan2
import panel as pn
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
pn.extension()

# Load dataset
df = pd.read_csv("dataset.csv")
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')
df = df.sort_values('Timestamp').reset_index(drop=True)

# Encode categorical features
vehicle_weight = {'bike': 0.5, 'car': 1.0, 'truck': 2.0}
df['VehicleWeight'] = df['VehicleType'].map(vehicle_weight).fillna(1.0)
traffic_weight = {'low': 1, 'medium': 2, 'high': 3}
df['TrafficLevel'] = df['TrafficConditionNearby'].map(traffic_weight)

In [14]:
# Model 2: Demand-Based Pricing
df['Demand'] = (
    1.5 * (df['Occupancy'] / df['Capacity']) +
    1.2 * df['QueueLength'] -
    0.8 * df['TrafficLevel'] +
    1.0 * df['IsSpecialDay'] +
    0.6 * df['VehicleWeight']
)
df['NormalizedDemand'] = (df['Demand'] - df['Demand'].min()) / (df['Demand'].max() - df['Demand'].min())
df['Price'] = 10 * (1 + 0.8 * df['NormalizedDemand'])
df['Price'] = df['Price'].clip(lower=5, upper=20)

# Model 3: Competitive Pricing
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat / 2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

def adjust_price_with_competition(row, timestamp_group):
    target_lat, target_lon = row['Latitude'], row['Longitude']
    target_price, target_occ, target_lot = row['Price'], row['Occupancy'], row['SystemCodeNumber']
    competitors = [
        (haversine(target_lat, target_lon, r['Latitude'], r['Longitude']), r['Price'], r['Occupancy'])
        for _, r in timestamp_group.iterrows()
        if r['SystemCodeNumber'] != target_lot and haversine(target_lat, target_lon, r['Latitude'], r['Longitude']) <= 1.0
    ]
    if not competitors:
        return target_price
    avg_p = np.mean([c[1] for c in competitors])
    avg_o = np.mean([c[2] for c in competitors])
    if target_occ > 0.9 * row['Capacity'] and avg_p < target_price:
        return max(target_price - 1.5, 5)
    elif avg_p > target_price and avg_o > 0.8 * row['Capacity']:
        return min(target_price + 1.5, 20)
    return target_price

# ✅ Fixed: Remove DeprecationWarning by using group_keys=False
df['FinalPrice'] = (
    df.groupby('Timestamp', group_keys=False)
      .apply(lambda g: g.apply(lambda r: adjust_price_with_competition(r, g), axis=1))
)

In [15]:
# 📊 Interactive Panel Dashboard
df_source = ColumnDataSource(df)
lot_options = df['SystemCodeNumber'].unique().tolist()
lot_selector = pn.widgets.Select(name='Select Parking Lot', options=lot_options)

@pn.depends(lot_selector)
def update_plot(selected_lot):
    lot_df = df[df['SystemCodeNumber'] == selected_lot].sort_values('Timestamp')
    source = ColumnDataSource(data={
        'Timestamp': lot_df['Timestamp'],
        'Model2Price': lot_df['Price'],
        'Model3Price': lot_df['FinalPrice']
    })
    p = figure(x_axis_type="datetime", title=f"Dynamic Pricing - {selected_lot}", height=400, width=800)
    p.line('Timestamp', 'Model2Price', source=source, legend_label='Model 2', color='blue', line_width=2)
    p.line('Timestamp', 'Model3Price', source=source, legend_label='Model 3', color='green', line_width=2)
    p.xaxis.axis_label = 'Time'
    p.yaxis.axis_label = 'Price (USD)'
    p.legend.location = 'top_left'
    return pn.pane.Bokeh(p)

dashboard = pn.Column("### 📊 Real-Time Pricing Viewer", lot_selector, update_plot)

# Display the dashboard directly in the notebook output
dashboard

Column
    [0] Markdown(str)
    [1] Select(options=['BHMBCCMKT01', ...], value='BHMBCCMKT01')
    [2] ParamFunction(function, _pane=Bokeh, defer_load=False)